<a href="https://colab.research.google.com/github/nickboard/Geospatial/blob/main/WaterExploratoryAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Define the file path
water_quality = '/content/Water_Quality_Site_Data_Cleaned.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(water_quality)

# Display the dataframe
df.head()



,OBJECTID,Slope_Gradient___Weighted_Average,Available_Water_Storage_0_25_cm___Weighted_Average,Available_Water_Storage_0_50_cm___Weighted_Average,Available_Water_Storage_0_100_cm___Weighted_Average,Available_Water_Storage_0_150_cm___Weighted_Average,Hydric_Classification___Presence,Component_Percentage___Representative_Value,Soil_Loss_Tolerance_Factor,Wind_Erodibility_Index,...,Saturated_Hydraulic_Conductivity,Conductivity,Lat,Long,distance_to_closest_populated,HUC8_Name,Month,Elevation,Temperature,pH
0,1,15.0,5.53,9.51,15.19,18.940001,4,80,3.0,86.0,...,9.00,135.50,44.100527,-111.172824,0.479030,5,7.0,1745.959961,16.60,NaN
1,2,48.0,3.25,6.50,13.00,16.510000,0,50,3.0,48.0,...,28.23,148.10,44.110000,-111.440000,0.128935,5,7.0,1575.508911,18.20,8.34
2,3,48.0,3.25,6.50,13.00,16.510000,0,50,3.0,48.0,...,28.23,142.95,44.110000,-111.440000,0.128935,5,10.0,1575.508911,13.00,8.34
3,4,48.0,3.25,6.50,13.00,16.510000,0,50,3.0,48.0,...,28.23,146.20,44.110000,-111.440000,0.128935,5,11.0,1575.508911,5.40,7.85
4,5,48.0,3.25,6.50,13.00,16.510000,0,50,3.0,48.0,...,28.23,155.95,44.110000,-111.440000,0.128935,5,11.0,1575.508911,2.05,8.02


In [ ]:
# Summary statistics for numerical columns
numerical_summary = df.describe()
numerical_summary

,OBJECTID,Slope_Gradient___Weighted_Average,Available_Water_Storage_0_25_cm___Weighted_Average,Available_Water_Storage_0_50_cm___Weighted_Average,Available_Water_Storage_0_100_cm___Weighted_Average,Available_Water_Storage_0_150_cm___Weighted_Average,Hydric_Classification___Presence,Component_Percentage___Representative_Value,Soil_Loss_Tolerance_Factor,Wind_Erodibility_Index,...,Saturated_Hydraulic_Conductivity,Conductivity,Lat,Long,distance_to_closest_populated,HUC8_Name,Month,Elevation,Temperature,pH
count,713.000000,361.000000,361.000000,361.000000,361.000000,361.000000,713.000000,713.000000,361.000000,361.000000,...,361.000000,705.000000,713.000000,713.000000,713.000000,713.0,616.000000,713.000000,615.000000,592.000000
mean,357.000000,12.418837,4.228670,7.158393,10.608975,13.656842,4.708275,81.746143,3.141274,63.883657,...,9.061136,171.493997,44.439397,-111.401285,0.149420,5.0,6.698052,1930.092698,13.001138,7.748851
std,205.969658,16.062520,0.686841,1.175168,2.536286,3.200450,16.202082,21.390269,1.045885,20.506849,...,9.188505,377.754271,0.109514,0.080489,0.146284,0.0,2.107608,69.752439,5.328469,0.490661
min,1.000000,1.000000,2.450000,5.490000,7.720000,8.850000,0.000000,31.000000,2.000000,0.000000,...,0.300000,0.000000,44.100527,-111.674333,0.004894,5.0,1.000000,1575.508911,0.050000,6.100000
25%,179.000000,2.000000,3.770000,6.780000,9.040000,11.090000,0.000000,60.000000,2.000000,56.000000,...,2.700000,125.000000,44.419722,-111.440000,0.087124,5.0,6.000000,1922.101074,10.475000,7.470000
50%,357.000000,8.800000,4.260000,6.910000,9.720000,13.540000,0.000000,95.000000,3.000000,56.000000,...,3.000000,139.000000,44.420000,-111.397222,0.095970,5.0,7.000000,1922.101074,13.600000,7.640000
75%,535.000000,10.200000,4.680000,7.390000,11.860000,15.160000,0.000000,100.000000,3.000000,86.000000,...,9.000000,173.000000,44.460000,-111.370000,0.165182,5.0,7.000000,1939.998535,17.550000,8.040000
max,713.000000,60.000000,9.010000,17.190001,27.379999,29.040001,100.000000,100.000000,5.000000,86.000000,...,28.230000,9999.000000,44.670636,-111.100000,0.785189,5.0,12.000000,2227.412598,22.500000,9.000000


In [ ]:

# Load the cleaned dataset
cleaned_df = pd.read_csv('Water_Quality_Site_Data_Cleaned.csv')

# Identify numeric and non-numeric columns
numeric_cols = cleaned_df.select_dtypes(include=['number']).columns
non_numeric_cols = cleaned_df.select_dtypes(exclude=['number']).columns

# Replace NaN values with the column's mean for numeric columns
cleaned_df[numeric_cols] = cleaned_df[numeric_cols].apply(lambda col: col.fillna(col.mean()))

# Remove non-numeric columns
cleaned_df = cleaned_df.drop(columns=non_numeric_cols)

# Save the cleaned dataset to a new CSV file
cleaned_df.to_csv('Water_Quality_Site_Data_Cleaned.csv', index=False)



In [ ]:
# Remove the 'OBJECTID' column
cleaned_df = df.drop('OBJECTID', axis=1)

# Calculate the correlation matrix
correlation_matrix = df.corr()

# Create a heatmap of the correlation matrix
plt.figure(figsize=(20, 16))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap of Water Quality Site Data')
plt.tight_layout()
plt.savefig('correlation_heatmap.png')
plt.close()

# Identify strong linear relationships
strong_correlations = (correlation_matrix.abs() > 0.7) & (correlation_matrix != 1.0)
strong_pairs = np.where(strong_correlations)
strong_correlations_list = [(correlation_matrix.index[x], correlation_matrix.columns[y], correlation_matrix.iloc[x, y]) for x, y in zip(*strong_pairs) if x < y]

print('Strong linear relationships (correlation > 0.7 or < -0.7):')
for pair in strong_correlations_list:
    print(f'{pair[0]} and {pair[1]}: {pair[2]:.2f}')

# Display summary statistics
print('Summary statistics:')
print(df.describe())

Strong linear relationships (correlation > 0.7 or < -0.7):
OBJECTID and Lat: 0.85
OBJECTID and Elevation: 0.72
Available_Water_Storage_0_25_cm___Weighted_Average and Available_Water_Storage_0_50_cm___Weighted_Average: 0.75
Available_Water_Storage_0_50_cm___Weighted_Average and Available_Water_Storage_0_100_cm___Weighted_Average: 0.85
Available_Water_Storage_0_50_cm___Weighted_Average and Available_Water_Storage_0_150_cm___Weighted_Average: 0.72
Available_Water_Storage_0_50_cm___Weighted_Average and Initial_Subsidence___Representative_Value: 0.78
Available_Water_Storage_0_100_cm___Weighted_Average and Available_Water_Storage_0_150_cm___Weighted_Average: 0.91
Available_Water_Storage_0_100_cm___Weighted_Average and Root_Zone_Available_Water_Storage___earthy_major_components: 0.89
Available_Water_Storage_0_150_cm___Weighted_Average and Root_Zone_Available_Water_Storage___earthy_major_components: 0.97
Component_Percentage___Representative_Value and Potential_Wetland_Soil_Landscapes: 0.84
Wi

In [ ]:
# Heatmap
# Compute the correlation matrix
corr_matrix = cleaned_df.corr()

# Generate a heatmap using Plotly
fig = px.imshow(corr_matrix,
                labels=dict(color="Correlation"),
                x=corr_matrix.columns,
                y=corr_matrix.columns,
                color_continuous_scale='RdBu_r',
                zmin=-1, zmax=1)

fig.update_layout(title='Correlation Heatmap', width=800, height=800)

fig.show()